<a href="https://colab.research.google.com/github/BhavikBuchke/Stock-price-Prediction/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install yfinance

In [ ]:
import yfinance as yf
from datetime import datetime, timedelta

# Define the ticker symbol for Reliance Industries (RIL) on NSE
ticker_symbol = "RELIANCE.NS"

# Calculate the start date (5 years ago from today)
end_date = datetime.today()
start_date = end_date - timedelta(days=10*365) # Approximating 10 years

# Fetch the data
reliance_data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Display the first few rows of the data
display(reliance_data.head())

# Save the data to a CSV file
reliance_data.to_csv('reliance_stock_data.csv')
print("\nReliance stock data saved to 'reliance_stock_data.csv'")

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Use the 'cleaned_reliance_data' DataFrame which is already available
# Select the 'Close' price column and convert it to a numpy array
close_prices = cleaned_reliance_data['Close'].values.reshape(-1, 1)

# Scale the data using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(close_prices)

# Create the training data set
X = []
Y = []
time_step = 60

for i in range(time_step, len(scaled_data)):
    X.append(scaled_data[i-time_step:i, 0])
    Y.append(scaled_data[i, 0])

# Convert X and Y to numpy arrays
X, Y = np.array(X), np.array(Y)

# Reshape X for the LSTM model (samples, time_steps, features)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

print("Shape of X:", X.shape)
print("Shape of Y:", Y.shape)

In [ ]:
# Split the data into training, validation, and testing sets
train_size = int(len(X) * 0.8)  # 80% for training
val_size = int(len(X) * 0.1)   # 10% for validation
test_size = len(X) - train_size - val_size # Remaining 10% for testing

X_train, X_val, X_test = X[:train_size], X[train_size:train_size+val_size], X[train_size+val_size:]
Y_train, Y_val, Y_test = Y[:train_size], Y[train_size:train_size+val_size], Y[train_size+val_size:]

print("Shape of X_train:", X_train.shape)
print("Shape of Y_train:", Y_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of Y_val:", Y_val.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of Y_test:", Y_test.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Build the LSTM model
model = Sequential()

# Add LSTM layers with Dropout
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50))
model.add(Dropout(0.2))

# Add the output layer
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Print the model summary
model.summary()

In [ ]:
# Train the model
history = model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_data=(X_val, Y_val))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt
import plotly.graph_objects as go

# Make predictions on the test data
predictions = model.predict(X_test)

# Inverse transform the predictions and the actual test data
predictions = scaler.inverse_transform(predictions)
Y_test_inverse = scaler.inverse_transform(Y_test.reshape(-1, 1))

# Calculate RMSE
rmse = sqrt(mean_squared_error(Y_test_inverse, predictions))
print(f"Root Mean Squared Error (RMSE): {rmse}")

In [ ]:
# Create an interactive plot using Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(len(Y_test_inverse))), y=Y_test_inverse.flatten(), mode='lines', name='Actual Price'))
fig.add_trace(go.Scatter(x=list(range(len(predictions))), y=predictions.flatten(), mode='lines', name='Predicted Price'))

fig.update_layout(
    title='Reliance Stock Price Prediction (Test Set)',
    xaxis_title='Time Step',
    yaxis_title='Close Price INR (₹)',
    legend_title='Price Type'
)

fig.show()